1.2.9 Método para análise estratégica: Análise estratégica de dados para alinhamento das outorgas portuárias

In [1]:
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
import pyomo.opt
import pyomo.environ as pe

In [2]:
origem = ["SP", "SC"]
destino = ["USA", "EU"]
porto1 = ["Itapoá-in", "Itapoá-out"]
porto2 = [ "SFS-in", "SFS-out"]

arcos = [(origem[0], porto1[0]), (origem[0], porto2[0]), (origem[1], porto1[0]), (origem[1], porto2[0]),
         (porto1[0], porto1[1]), (porto2[0], porto2[1]),
         (porto1[1], destino[0]), (porto2[1], destino[0]), (porto1[1], destino[1]), (porto2[1], destino[1])]
vertices = list(set().union(origem,destino,porto1,porto2))

transport_cost = {arcos[0]:.505,
                 arcos[1]:.190,
                 arcos[2]:.365,
                 arcos[3]:.265,
                 arcos[4]:.002,
                 arcos[5]:.001,
                 arcos[6]:.345,
                 arcos[7]:.525,
                 arcos[8]:.210,
                 arcos[9]:.245}

links_capacity = {arcos[0]:1,
                 arcos[1]:1,
                 arcos[2]:1,
                 arcos[3]:1,
                 arcos[4]:2,
                 arcos[5]:2,
                 arcos[6]:1,
                 arcos[7]:1,
                 arcos[8]:1,
                 arcos[9]:1}

origem_destino = [(o,d) for o in origem for d in destino]#commodities
demanda_origem_destino = {  origem_destino[0]:1, 
                            origem_destino[1]:1,
                            origem_destino[2]:1, 
                            origem_destino[3]:1 }

# print("link => custo")
# for key, value in transportCost.items():
#     print(key,'=>', value)
# print()
# print("link => capacidade")    
# for key, value in linksCapacity.items():
#     print(key,'=>', value)
demanda_origem_destino


{('SP', 'USA'): 1, ('SP', 'EU'): 1, ('SC', 'USA'): 1, ('SC', 'EU'): 1}

Muticomodity Minimum cost flow problem

In [3]:
array = set()
for e in arcos:
    for od in origem_destino:
        if e[0]==od[0] or e[1]==od[1]:
            array.add(((*e, *od)))
print(array.pop())

('SP', 'Itapoá-in', 'SP', 'USA')


In [4]:
array = set()
for n in vertices:
    for od in origem_destino:
        if n==od[0] or n==od[1]:
            array.add((n, od))
print(array)

{('USA', ('SC', 'USA')), ('SC', ('SC', 'EU')), ('EU', ('SP', 'EU')), ('SC', ('SC', 'USA')), ('SP', ('SP', 'EU')), ('SP', ('SP', 'USA')), ('USA', ('SP', 'USA')), ('EU', ('SC', 'EU'))}


In [41]:
modelo = pe.ConcreteModel(name="(demanda_porto)")
                  
modelo.dual = pe.Suffix(direction=pe.Suffix.IMPORT)
modelo.node_set = pe.Set( initialize=vertices )
modelo.edge_set = pe.Set( initialize=arcos, dimen=2)
modelo.commodity_set = pe.Set( initialize=origem_destino, dimen=2)

modelo.var_indexes = pe.Set(initialize=modelo.edge_set*modelo.commodity_set)
modelo.var = pe.Var(modelo.var_indexes, within=pe.NonNegativeReals)

def objetivo(m):
    return sum(transport_cost[(index[0], index[1])] * m.var[index] for index in modelo.var_indexes)     
modelo.obj = pe.Objective(rule=objetivo, sense=pe.minimize)

def conserva_fluxo(m, i, o, d):
    from_i = [v for v in arcos if v[0] == i]
    to_i = [v for v in arcos if v[1] == i]
    q =  demanda_origem_destino[(o,d)] if i == o else -demanda_origem_destino[(o,d)] if i == d else 0
    return sum(m.var[v,(o,d)] for v in from_i) - sum(m.var[v,(o,d)] for v in to_i) == q

modelo.conserva_fluxo_indexes = pe.Set(initialize=modelo.node_set*modelo.commodity_set)
modelo.flowConservation = pe.Constraint(modelo.conserva_fluxo_indexes, rule=conserva_fluxo)

def capacidades(m, e1,e2):
    return sum(m.var[e1,e2,o,d] for (o,d) in modelo.commodity_set) <= links_capacity[(e1,e2)]
modelo.capacidade_arco = pe.Constraint(modelo.edge_set, rule=capacidades)
 
solver = pyomo.opt.SolverFactory('cbc')
results = solver.solve(modelo)
#print(results)
# print(results.solver.status)
# modelo.x.pprint()
# modelo.dual.display()
#print(objetivo(modelo))
#print(conserva_fluxo(modelo, vertices, origem_destino))
#modelo.display()
#modelo.pprint()
for var_index in modelo.var_indexes:
    print(var_index,modelo.var[var_index].value)
# for con in modelo.component_map(pe.Constraint).itervalues():
#     print(con.pprint())

('SC', 'SFS-in', 'SC', 'USA') 0.0
('SP', 'SFS-in', 'SC', 'USA') 0.0
('SC', 'SFS-in', 'SC', 'EU') 1.0
('SFS-out', 'EU', 'SP', 'EU') 0.0
('SFS-in', 'SFS-out', 'SP', 'EU') 0.0
('SC', 'Itapoá-in', 'SP', 'EU') 0.0
('SFS-out', 'USA', 'SC', 'USA') 0.0
('SFS-out', 'EU', 'SP', 'USA') 0.0
('SFS-in', 'SFS-out', 'SP', 'USA') 1.0
('SFS-out', 'USA', 'SP', 'USA') 1.0
('SP', 'Itapoá-in', 'SC', 'EU') 0.0
('Itapoá-out', 'EU', 'SC', 'USA') 0.0
('SFS-in', 'SFS-out', 'SC', 'USA') 0.0
('Itapoá-in', 'Itapoá-out', 'SC', 'EU') 0.0
('SFS-in', 'SFS-out', 'SC', 'EU') 1.0
('Itapoá-out', 'USA', 'SP', 'USA') 0.0
('Itapoá-in', 'Itapoá-out', 'SP', 'EU') 1.0
('Itapoá-out', 'USA', 'SC', 'USA') 1.0
('Itapoá-out', 'EU', 'SP', 'EU') 1.0
('SP', 'Itapoá-in', 'SP', 'USA') 0.0
('SFS-out', 'EU', 'SC', 'USA') 0.0
('SFS-out', 'EU', 'SC', 'EU') 1.0
('Itapoá-out', 'EU', 'SP', 'USA') 0.0
('SC', 'Itapoá-in', 'SP', 'USA') 0.0
('SP', 'SFS-in', 'SC', 'EU') 0.0
('SC', 'SFS-in', 'SP', 'EU') 0.0
('SC', 'SFS-in', 'SP', 'USA') 0.0
('SP', 'It

In [6]:
for k in modelo.indexes:
    print(k[0],k)

SC ('SC', 'SFS-in', 'SC', 'USA')
SP ('SP', 'SFS-in', 'SC', 'USA')
SC ('SC', 'SFS-in', 'SC', 'EU')
SFS-out ('SFS-out', 'EU', 'SP', 'EU')
SFS-in ('SFS-in', 'SFS-out', 'SP', 'EU')
SC ('SC', 'Itapoá-in', 'SP', 'EU')
SFS-out ('SFS-out', 'USA', 'SC', 'USA')
SFS-out ('SFS-out', 'EU', 'SP', 'USA')
SFS-in ('SFS-in', 'SFS-out', 'SP', 'USA')
SFS-out ('SFS-out', 'USA', 'SP', 'USA')
SP ('SP', 'Itapoá-in', 'SC', 'EU')
Itapoá-out ('Itapoá-out', 'EU', 'SC', 'USA')
SFS-in ('SFS-in', 'SFS-out', 'SC', 'USA')
Itapoá-in ('Itapoá-in', 'Itapoá-out', 'SC', 'EU')
SFS-in ('SFS-in', 'SFS-out', 'SC', 'EU')
Itapoá-out ('Itapoá-out', 'USA', 'SP', 'USA')
Itapoá-in ('Itapoá-in', 'Itapoá-out', 'SP', 'EU')
Itapoá-out ('Itapoá-out', 'USA', 'SC', 'USA')
Itapoá-out ('Itapoá-out', 'EU', 'SP', 'EU')
SP ('SP', 'Itapoá-in', 'SP', 'USA')
SFS-out ('SFS-out', 'EU', 'SC', 'USA')
SFS-out ('SFS-out', 'EU', 'SC', 'EU')
Itapoá-out ('Itapoá-out', 'EU', 'SP', 'USA')
SC ('SC', 'Itapoá-in', 'SP', 'USA')
SP ('SP', 'SFS-in', 'SC', 'EU')
SC

In [7]:
vakkkk =modelo.conserva_fluxo_indexes
print(vakkkk.dimen)
print(modelo.commodity_set.dimen)
for i in vakkkk:
    print(i)

3
2
('Itapoá-in', 'SC', 'EU')
('Itapoá-out', 'SP', 'EU')
('SC', 'SP', 'EU')
('SP', 'SC', 'EU')
('SP', 'SC', 'USA')
('SFS-out', 'SP', 'USA')
('EU', 'SP', 'USA')
('USA', 'SC', 'USA')
('SFS-in', 'SP', 'EU')
('SFS-out', 'SC', 'USA')
('SC', 'SC', 'EU')
('USA', 'SP', 'EU')
('USA', 'SP', 'USA')
('SFS-out', 'SC', 'EU')
('SC', 'SC', 'USA')
('Itapoá-out', 'SC', 'USA')
('Itapoá-in', 'SP', 'USA')
('SFS-in', 'SC', 'EU')
('EU', 'SC', 'USA')
('SP', 'SP', 'EU')
('SFS-in', 'SC', 'USA')
('EU', 'SC', 'EU')
('Itapoá-in', 'SP', 'EU')
('Itapoá-out', 'SC', 'EU')
('Itapoá-in', 'SC', 'USA')
('Itapoá-out', 'SP', 'USA')
('SP', 'SP', 'USA')
('SFS-out', 'SP', 'EU')
('SFS-in', 'SP', 'USA')
('SC', 'SP', 'USA')
('EU', 'SP', 'EU')
('USA', 'SC', 'EU')
